## EDA - Brasileirão Pontos Corridos (2003 - 2024)


# Objetivo

Essa análise tem como objetivo obter mais detalhes e conhecer mais os dados do Campeonato Brasileiro de Futebol desde que passou a ser disputado no formato de pontos corridos, em 2003, até a última edição concluída, em 2024. Alguns dos objetivos são:

Identificar os times que participaram do campeonato e os respectivos aproveitamentos, tanto geral quanto como visitante / mandante. 

Identificar as melhores campanhas, a quantidade de títulos, rebaixamentos e saldos de gols dos times. 

Ver as participações dos clubes no Brasileirão por estado e por região.